<a href="https://colab.research.google.com/github/fantajeon/DLPytorch1.2/blob/master/Chapter1_nn_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#nn.Module로 모델 감싸기#
이번 강의는 $\hat{y} = wx + b$을 직접적으로 계산하였다면, PyTorch에서 권장하는 nn.Module로 만드는 과정을 배워보겠습니다. 보통 작게 할때는 별 문제가 되지 않지만, 큰 모델을 설계할때는 시스템의 힘을 빌려야 합니다. nn.Module을 사용하면, 모델의 파라미터 추출 및 내부 저장, 모듈화, 기존의 잘 알려진 강력한 모듈을 쉽게 가져다 쓸수 있습니다. 즉, 많은 이점이 있습니다. 자주 쓰시다 보면 느끼게 되실 겁니다.

In [0]:
# 본 강좌에서 필요한 패키지를 Python에 import 합니다.
import torch
from torch import nn

# $wx+b$를 nn.Module화 하기 #
nn.Module화를 하려면 다음의 3가지 기능을 최소로 구현해야 합니다.
1. nn.Module로 상속하기
2. \__init__()
3. forward() 정의


**nn.Module**은 PyTorch에서 Module을 대표하는 가장 기초 클래스입니다.

**\__init__()**은 클래스를 초기화 하는 함수입니다. 보통 모델에 필요한 변수들을 정의합니다.

**forward()**는 Computation Graph에 기록할 연산 과정을 이 함수에서 실제로 구현합니다. 그리고 모듈 밖에서 보면 프로그래밍에서 의미하는 함수를 호출하는 거와 비슷합니다. 예를들면 다음 코드와 같습니다:
``` python
# Python 함수 정의
def f(x): 
  return wx + b
y = f(x)   # 함수 호출하면 변수 y에 결과값 저장
```

아래 코드에서 nn.Parameter()가 있습니다. 모델의 인자를 정의하는 함수인데, 이것은 torch.tensor와 같습니다. 그런데 다른 한가지 기능이 더 있습니다. 이 함수는 nn.Module과 긴밀히 상호작용을 합니다. 예를들면, 자동으로 "나는 이 클래스의 인자!"야라고 자동으로 인자 리스트에 추가됩니다. 이로 인하여 향후에 PyTorch에서 제공하는 최적화 패키지들과 쉽게 연동할 수 있습니다. 

In [9]:
#nn.Module화 하기
class Model(nn.Module): # 1. 상속
  def __init__(self):   # 2. 모듈 초기화 구현
    super(Model, self).__init__()
    self.w = nn.Parameter( torch.rand(1, dtype=torch.float) )   # 모델의 w 인자 정의(이런게 있군요 눈으로만 여겨 보자!)
    self.b = nn.Parameter( torch.rand(1, dtype=torch.float) )   # 모델의 b 인자 정의

  def forward(self, x): # 3. forward()구현
    return self.w * x + self.b

# 테스트 코드
f = Model()   # 이제 Module의 클래스를 인스턴스화 합니다.
test_x = torch.rand(3, dtype=torch.float32)
print( "f(x) = ", f(test_x) )


f(x) =  tensor([0.2009, 0.2027, 0.2040], grad_fn=<AddBackward0>)


## 모델의 파라미터를 확인해 보자 ##
nn.Parameter()로 모델에 자동등록 된다고 했습니다. 그러면 어떻게 등록되었는지를 확인해 보겠습니다. 보통은 named_parameter()로 호출합니다.
### nn.Parameter() ###
이 함수는 2개의 인자를 받습니다:
1. data: 하나는 변수에 채월질 초기화 데이터
2. requires_grad: backward시에 미분값을 계산한다
  * 기본은 True로 소유한다
  * False면 소유하지 않는다.

### nn.Module.named_parameters() ###
모델의 인자 리스트를 변수명과 함께 같이 반환한다. 
다음과 같이 2개의 인자를 가지고 있습니다:
1. prefix: 접두어를 붙인다. 기본은 공백이다.
2. recurse: Module내에 서브 Module까지의 인자를 모두 가져온다.
  * 기본은 True로 모두 가져온다.
백문이 불여 일견입니다. 한번 코드를 실행해겠습니다.


In [25]:
mymodel = Model()

print("####  기본값 사용")
for name, param in mymodel.named_parameters():
  print("=> var name", name)
  print("=> param", "data:", param.data, "size:", param.size(), "dim:", param.dim())

print("####   prefix=aa 사용")
for name, param in mymodel.named_parameters(prefix="aa"):
  print("=> var name", name)
  print("=> param", "data:", param.data, "size:", param.size(), "dim:", param.dim())



####  기본값 사용
=> var name w
=> param data: tensor([0.5923]) size: torch.Size([1]) dim: 1
=> var name b
=> param data: tensor([0.9532]) size: torch.Size([1]) dim: 1
####   prefix=aa 사용
=> var name aa.w
=> param data: tensor([0.5923]) size: torch.Size([1]) dim: 1
=> var name aa.b
=> param data: tensor([0.9532]) size: torch.Size([1]) dim: 1


In [29]:
# 서브 모듈을 강제로 만들어 보겠습니다.
class Temp(nn.Module):
  def __init__(self):
    super(Temp, self).__init__()
    self.z = nn.Parameter(torch.rand(1), requires_grad=True)
    self.f = Model()

  def forward(self, x):
    return self.f(x) + self.z

print("####   recurse=True 사용")
g = Temp()
for name, param in g.named_parameters(recurse=True):
  print("=> var name", name)
  print("=> param", "data:", param.data, "size:", param.size(), "dim:", param.dim())
  
print("####   recurse=False 사용")
for name, param in g.named_parameters(recurse=False):
  print("=> var name", name)
  print("=> param", "data:", param.data, "size:", param.size(), "dim:", param.dim())

####   recurse=True 사용
=> var name z
=> param data: tensor([0.7470]) size: torch.Size([1]) dim: 1
=> var name f.w
=> param data: tensor([0.0072]) size: torch.Size([1]) dim: 1
=> var name f.b
=> param data: tensor([0.3880]) size: torch.Size([1]) dim: 1
####   recurse=False 사용
=> var name z
=> param data: tensor([0.7470]) size: torch.Size([1]) dim: 1


#Gradient Descendent Optimization에 적용#
직접적으로 $y = wx + b$를 했다면, 이젠 Model 클래스의 인스턴스로 바꿔보겠습니다. 이제는 w, b를 직접 지정하지 않았고, named_parameters와 for 루프의 조합으로 끝을 냈습니다.

\* 참고로 gradient는 각 변수의 어느 쪽으로 변화하는 지를 가지고 있는 백터입니다.

In [33]:
x = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6], dtype=torch.float32)
y = torch.tensor([15, 25, 40, 55, 65, 66], dtype=torch.float32)


def loss(pred_y, y):
  return 0.5*(pred_y - y).pow(2.0).sum()

learning_rate = 0.0001

model = Model() # 모델 인스턴스화
for step in range(300000):
  # PyTorch는 Dynamic하게 연산을 기록하는 기능 때문에, 
  # 매번 loop에서 x--> predict -> loss-->backward()-->gradient descent를 해줘야 함.
  pred_y = model(x)
  L = loss(pred_y, y)
  model.zero_grad() # gradient[편미분을 모아 놓은 벡터] 초기화
  L.backward()
  # torch.no_grad()는 computation graph에서 backward()시 제외된다. 예측만 할때 유용하다.
  with torch.no_grad():
    for name, param in model.named_parameters():
      param -= learning_rate * param.grad

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={:.3}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))


Step:0, L:6858.2, w=0.751, b=0.355, grad(w)=-1.11e+02, grad(b)=-2.62e+02
Step:10000, L:643.36, w=28.9, b=34.9, grad(w)=-12.9, grad(b)=4.3
Step:20000, L:483.53, w=40.8, b=30.7, grad(w)=-11.0, grad(b)=3.95
Step:30000, L:366.41, w=50.9, b=27.1, grad(w)=-9.4, grad(b)=3.38
Step:40000, L:280.59, w=59.6, b=23.9, grad(w)=-8.05, grad(b)=2.89
Step:50000, L:217.7, w=67.1, b=21.3, grad(w)=-6.89, grad(b)=2.48
Step:60000, L:171.62, w=73.5, b=19.0, grad(w)=-5.9, grad(b)=2.12
Step:70000, L:137.85, w=78.9, b=17.0, grad(w)=-5.05, grad(b)=1.81
Step:80000, L:113.11, w=83.6, b=15.3, grad(w)=-4.32, grad(b)=1.55
Step:90000, L:94.972, w=87.6, b=13.9, grad(w)=-3.7, grad(b)=1.33
Step:100000, L:81.684, w=91.1, b=12.7, grad(w)=-3.17, grad(b)=1.14
Step:110000, L:71.948, w=94.0, b=11.6, grad(w)=-2.71, grad(b)=0.976
Step:120000, L:64.813, w=96.5, b=10.7, grad(w)=-2.32, grad(b)=0.835
Step:130000, L:59.588, w=98.6, b=9.93, grad(w)=-1.99, grad(b)=0.712
Step:140000, L:55.755, w=1e+02, b=9.27, grad(w)=-1.7, grad(b)=0.611

# 전체 코드를 하나로 모은 내용 #

In [34]:
#nn.Module화 하기
class Model(nn.Module): # 1. 상속
  def __init__(self):   # 2. 모듈 초기화 구현
    super(Model, self).__init__()
    self.w = nn.Parameter( torch.rand(1, dtype=torch.float) )   # 모델의 w 인자 정의(이런게 있군요 눈으로만 여겨 보자!)
    self.b = nn.Parameter( torch.rand(1, dtype=torch.float) )   # 모델의 b 인자 정의

  def forward(self, x): # 3. forward()구현
    return self.w * x + self.b

def loss(pred_y, y):
  return 0.5*(pred_y - y).pow(2.0).sum()

learning_rate = 0.0001

x = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6], dtype=torch.float32)
y = torch.tensor([15, 25, 40, 55, 65, 66], dtype=torch.float32)

model = Model() # 모델 인스턴스화
for step in range(300000):
  # PyTorch는 Dynamic하게 연산을 기록하는 기능 때문에, 
  # 매번 loop에서 x--> predict -> loss-->backward()-->gradient descent를 해줘야 함.
  pred_y = model(x)
  L = loss(pred_y, y)
  model.zero_grad() # gradient[편미분을 모아 놓은 벡터] 초기화
  L.backward()
  # torch.no_grad()는 computation graph에서 backward()시 제외된다. 예측만 할때 유용하다.
  with torch.no_grad():
    for name, param in model.named_parameters():
      param -= learning_rate * param.grad

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={:.3}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))


Step:0, L:6839.3, w=0.881, b=0.372, grad(w)=-1.11e+02, grad(b)=-2.62e+02
Step:10000, L:642.0, w=29.0, b=34.9, grad(w)=-12.9, grad(b)=4.29
Step:20000, L:482.53, w=40.8, b=30.7, grad(w)=-11.0, grad(b)=3.94
Step:30000, L:365.68, w=51.0, b=27.0, grad(w)=-9.39, grad(b)=3.38
Step:40000, L:280.06, w=59.7, b=23.9, grad(w)=-8.04, grad(b)=2.89
Step:50000, L:217.31, w=67.1, b=21.2, grad(w)=-6.88, grad(b)=2.47
Step:60000, L:171.34, w=73.5, b=19.0, grad(w)=-5.89, grad(b)=2.12
Step:70000, L:137.64, w=79.0, b=17.0, grad(w)=-5.04, grad(b)=1.81
Step:80000, L:112.95, w=83.7, b=15.3, grad(w)=-4.32, grad(b)=1.55
Step:90000, L:94.86, w=87.7, b=13.9, grad(w)=-3.7, grad(b)=1.33
Step:100000, L:81.602, w=91.1, b=12.6, grad(w)=-3.16, grad(b)=1.14
Step:110000, L:71.887, w=94.0, b=11.6, grad(w)=-2.71, grad(b)=0.976
Step:120000, L:64.769, w=96.5, b=10.7, grad(w)=-2.32, grad(b)=0.834
Step:130000, L:59.555, w=98.7, b=9.92, grad(w)=-1.99, grad(b)=0.711
Step:140000, L:55.731, w=1e+02, b=9.26, grad(w)=-1.7, grad(b)=0.6